### With this high-risk solution approach we try to overcome the challenges faced when we try to forecast on a large dataset which contain forecast groups which is Counties in this case. Also, the statistical models used in medium risk solution cannot seemlessly integrate multivariate data for forecasting.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import lightgbm as lgb
import optuna

In [2]:
# Mount the drive
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Read Data

In [3]:
train_df = pd.read_csv('drive/MyDrive/DS5500/train.csv', index_col="row_id")
test_df = pd.read_csv('drive/MyDrive/DS5500/test.csv', index_col="row_id")
census_data = pd.read_csv('drive/MyDrive/DS5500/census_starter.csv', index_col='cfips')

In [4]:
train_df.head()

,cfips,county,state,first_day_of_month,microbusiness_density,active
row_id,,,,,,
1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [5]:
test_df.head()

,cfips,first_day_of_month
row_id,,
1001_2022-11-01,1001,2022-11-01
1003_2022-11-01,1003,2022-11-01
1005_2022-11-01,1005,2022-11-01
1007_2022-11-01,1007,2022-11-01
1009_2022-11-01,1009,2022-11-01


In [6]:
census_data.head()

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,pct_college_2021,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
cfips,,,,,,,,,,,,,,,,,,,,,
1001,76.6,78.9,80.6,82.7,85.5,14.5,15.9,16.1,16.7,16.4,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1003,74.5,78.1,81.8,85.1,87.9,20.4,20.7,21.0,20.2,20.6,...,1.4,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0
1005,57.2,60.4,60.5,64.6,64.6,7.6,7.8,7.6,7.3,6.7,...,0.5,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0
1007,62.0,66.1,69.2,76.1,74.6,8.1,7.6,6.5,7.4,7.9,...,1.2,1.4,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0
1009,65.8,68.5,73.0,79.6,81.0,8.7,8.1,8.6,8.9,9.3,...,1.3,1.4,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0


## Data Preparation/Feature Engineering

As we will be using ensemble models which perform regression the time-series data need to be prepared into a data suitable for regression. In this section we aim to achieve that by Data Preparation and Feature Engineering

In [7]:
# Add missing states to test_df
state_dict = train_df[['cfips', 'state', 'county']]
state_dict = state_dict.set_index('cfips')
state_dict = state_dict.drop_duplicates()
state_dict = state_dict.to_dict()

test_df['state'] = test_df['cfips'].map(state_dict['state'])
test_df['county'] = test_df['cfips'].map(state_dict['county'])

test_df.head()

,cfips,first_day_of_month,state,county
row_id,,,,
1001_2022-11-01,1001,2022-11-01,Alabama,Autauga County
1003_2022-11-01,1003,2022-11-01,Alabama,Baldwin County
1005_2022-11-01,1005,2022-11-01,Alabama,Barbour County
1007_2022-11-01,1007,2022-11-01,Alabama,Bibb County
1009_2022-11-01,1009,2022-11-01,Alabama,Blount County


Extract Features

In [8]:
# Variables
time_column = "first_day_of_month"
target_column = "microbusiness_density"
LAGS = [1, 2, 3, 4, 5, 6]

**Extract Time Features**

In [9]:
def time_featurizer(df, time_column):
  df[time_column] = pd.to_datetime(df[time_column])
  df['year'] = df[time_column].dt.year
  df['month'] = df[time_column].dt.month
  df['scale'] = (df[time_column] - df[time_column].min()).dt.days
  df['scale'] = df['scale'].factorize()[0]
  return df

In [10]:
df_all = pd.concat([train_df, test_df], axis=0)

df_all = time_featurizer(df_all, time_column)


In [11]:
df_all.head()

,cfips,county,state,first_day_of_month,microbusiness_density,active,year,month,scale
row_id,,,,,,,,,
1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249.0,2019,8,0
1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198.0,2019,9,1
1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269.0,2019,10,2
1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243.0,2019,11,3
1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243.0,2019,12,4


**Extract Lag Features**

In [12]:
def lag_featurizer(df, lags, target_column):
  for i in lags:
    df_all[f'lags({i})'] = df_all.groupby('cfips')[target_column].shift(i)
  
  df_all['active'] = df_all.groupby('cfips')['active'].shift(8)
  return df_all

In [13]:
df_all = lag_featurizer(df_all, LAGS, target_column)

In [14]:
df_all.head()

,cfips,county,state,first_day_of_month,microbusiness_density,active,year,month,scale,lags(1),lags(2),lags(3),lags(4),lags(5),lags(6)
row_id,,,,,,,,,,,,,,,
1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,NaN,2019,8,0,NaN,NaN,NaN,NaN,NaN,NaN
1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,NaN,2019,9,1,3.007682,NaN,NaN,NaN,NaN,NaN
1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,NaN,2019,10,2,2.884870,3.007682,NaN,NaN,NaN,NaN
1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,NaN,2019,11,3,3.055843,2.884870,3.007682,NaN,NaN,NaN
1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,NaN,2019,12,4,2.993233,3.055843,2.884870,3.007682,NaN,NaN


Handle Categorical Data

In [15]:
# Variables

categorical_columns = ["county", "state"]


Drop time column. We extracted this information using time_featurizer and we can't keep datetime datatype in a regression model

In [16]:
df_all = df_all.drop(columns=[time_column])

The advantage of using Ensemble Models like XGBoost or LGBM is that unlike Linear Regression they can handle categorical data within the model itself without the need for encoding (one-hot or label).

Source: https://xgboost.readthedocs.io/en/stable/tutorials/categorical.html

In [17]:
df_all[categorical_columns] = df_all[categorical_columns].astype('category')

In [18]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 147345 entries, 1001_2019-08-01 to 56045_2023-06-01
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   cfips                  147345 non-null  int64   
 1   county                 147345 non-null  category
 2   state                  147345 non-null  category
 3   microbusiness_density  122265 non-null  float64 
 4   active                 122265 non-null  float64 
 5   year                   147345 non-null  int64   
 6   month                  147345 non-null  int64   
 7   scale                  147345 non-null  int64   
 8   lags(1)                122265 non-null  float64 
 9   lags(2)                122265 non-null  float64 
 10  lags(3)                122265 non-null  float64 
 11  lags(4)                122265 non-null  float64 
 12  lags(5)                122265 non-null  float64 
 13  lags(6)                122265 non-null  float64 
dtypes

### Enrich the data with Census Data 

In [19]:
df_all = df_all.reset_index()
df_all = df_all.set_index('cfips')

df_all[census_data.columns] = census_data

df_all = df_all.reset_index()
df_all = df_all.set_index("row_id")

In [20]:
df_all.head()

,cfips,county,state,microbusiness_density,active,year,month,scale,lags(1),lags(2),...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
row_id,,,,,,,,,,,,,,,,,,,,,
1001_2019-08-01,1001,Autauga County,Alabama,3.007682,NaN,2019,8,0,NaN,NaN,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1001_2019-09-01,1001,Autauga County,Alabama,2.884870,NaN,2019,9,1,3.007682,NaN,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1001_2019-10-01,1001,Autauga County,Alabama,3.055843,NaN,2019,10,2,2.884870,3.007682,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1001_2019-11-01,1001,Autauga County,Alabama,2.993233,NaN,2019,11,3,3.055843,2.884870,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1001_2019-12-01,1001,Autauga County,Alabama,2.993233,NaN,2019,12,4,2.993233,3.055843,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0


Since we scaled the Date information into an equivalent scale we can't keep the first month of data since a scale value of 0 might bias the model

In [21]:
df_all[["year","month","scale"]].reset_index(drop=True).drop_duplicates()

,year,month,scale
0,2019,8,0
1,2019,9,1
2,2019,10,2
3,2019,11,3
4,2019,12,4
5,2020,1,5
6,2020,2,6
7,2020,3,7
8,2020,4,8
9,2020,5,9


In [22]:
df_all = df_all[df_all['scale'] != 0]

### Model Building

In [23]:
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true != 0) | (y_pred != 0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)

In [24]:
def to_percent(X, y):
    yhat = y / X['lags(1)']
    yhat[X['lags(1)'] == 0] = 0 # denominator cannot be 0
    return yhat

def from_percent(X, y):
    yhat = y * X[f'lags(1)']
    return yhat


**Ensemble Model used: LGBM**

LightGBM is a gradient boosting framework that uses tree based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.

* Lower memory usage.

* Better accuracy.

* Support of parallel, distributed, and GPU learning.

* Capable of handling large-scale data.

Source: https://lightgbm.readthedocs.io/en/v3.3.2/


**Hyperparameter Tuning Tool: Optuna**

Optuna is an automatic hyperparameter optimization software framework, particularly designed for machine learning. It features an imperative, define-by-run style user API. Thanks to our define-by-run API, the code written with Optuna enjoys high modularity, and the user of Optuna can dynamically construct the search spaces for the hyperparameters.

Source: https://optuna.org/


In [25]:
SMAPE_ENABLED = True

In [26]:
def lgbm_objective(trial):
    params = {
        'n_iter'           : 200,
        'verbosity'        : -1,
        'objective'        : 'l1',
        'random_state'     : 42,
        'extra_trees'      : True,
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode' : trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'max_depth'        : trial.suggest_int('max_depth', 3, 10),
        'learning_rate'    : trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'lambda_l1'        : trial.suggest_float('lambda_l1', 1e-2, 10.0),
        'lambda_l2'        : trial.suggest_float('lambda_l2', 1e-2, 10.0),
        'num_leaves'       : trial.suggest_int('num_leaves', 8, 1024),
        'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 5, 250),}
    
    model  = lgb.LGBMRegressor(**params)
    X, y   = df_all.drop(columns=[target_column]), df_all[target_column]
    
    train_times = list(range(38))
    valid_times = [38]
    
    y_train = y[X['scale'].isin(train_times)]
    y_valid = y[X['scale'].isin(valid_times)]
    
    X_train = X[X['scale'].isin(train_times)]
    X_valid = X[X['scale'].isin(valid_times)]
    
    if SMAPE_ENABLED:
        y_train = to_percent(X_train, y_train)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    
    if SMAPE_ENABLED:
        y_pred = from_percent(X_valid, y_pred)
    
    return smape(y_valid, y_pred)

In [27]:
study = optuna.create_study(direction='minimize', study_name='Regressor')
study.optimize(lgbm_objectivstudy.best_valuee, n_trials=30, show_progress_bar=True)

[I 2023-02-15 14:25:07,758] A new study created in memory with name: Regressor


  0%|          | 0/30 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=4.167584308535034, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.167584308535034
[LightGBM] [Warning] lambda_l2 is set=3.426548482200029, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.426548482200029
[LightGBM] [Warning] min_data_in_leaf is set=161, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=161
[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200
[I 2023-02-15 14:25:24,487] Trial 0 finished with value: 1.069598131192116 and parameters: {'colsample_bytree': 0.8592782063536475, 'colsample_bynode': 0.5221729924814734, 'max_depth': 7, 'learning_rate': 0.02908600318519419, 'lambda_l1': 4.167584308535034, 'lambda_l2': 3.426548482200029, 'num_leaves': 769, 'min_data_in_leaf': 161}. Best is trial 0 with value: 1.069598131192116.
[LightGBM] [Warning] lambda_l1 is set=3.1695907542486097, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.16959

This SOTA hyperparameter tuning framework ran through multiple iterations and selected the best parameters for us

In [30]:
study.best_params

{'colsample_bytree': 0.6664817732972992,
 'colsample_bynode': 0.8413937131649727,
 'max_depth': 5,
 'learning_rate': 0.04184232428767455,
 'lambda_l1': 0.027146051246965897,
 'lambda_l2': 4.2111398913050975,
 'num_leaves': 622,
 'min_data_in_leaf': 207}

Now, we will make predictions using this an LGBM model that uses the optimal parameters

In [32]:
model = lgb.LGBMRegressor(**study.best_params)

In [33]:
model

LGBMRegressor(colsample_bynode=0.8413937131649727,
              colsample_bytree=0.6664817732972992,
              lambda_l1=0.027146051246965897, lambda_l2=4.2111398913050975,
              learning_rate=0.04184232428767455, max_depth=5,
              min_data_in_leaf=207, num_leaves=622)

In [59]:
X, y   = df_all.drop(columns=[target_column]), df_all[target_column]
X_test, y_test = X[y.isnull()], y[y.isnull()]

train_times = list(range(38))

X_train = X[X['scale'].isin(train_times)]
y_train = y[X['scale'].isin(train_times)]

if SMAPE_ENABLED:
    y_train = to_percent(X_train, y_train)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

if SMAPE_ENABLED:
    y_pred = from_percent(X_test, y_pred)


[LightGBM] [Warning] lambda_l1 is set=0.027146051246965897, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.027146051246965897
[LightGBM] [Warning] min_data_in_leaf is set=207, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=207
[LightGBM] [Warning] lambda_l2 is set=4.2111398913050975, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.2111398913050975


Now that we have done the predictions let's add it back to the test dataframe 

In [54]:
# Placeholder for target column in test dataset
test_df[target_column] = 0


In [61]:
# Replace the target column with the predicted values from LGBM
test_df.loc[y_pred.index, target_column] = y_pred


In [62]:
test_df.head()

,cfips,first_day_of_month,state,county,microbusiness_density
row_id,,,,,
1001_2022-11-01,1001,2022-11-01,Alabama,Autauga County,3.470966
1003_2022-11-01,1003,2022-11-01,Alabama,Baldwin County,8.375717
1005_2022-11-01,1005,2022-11-01,Alabama,Barbour County,1.236247
1007_2022-11-01,1007,2022-11-01,Alabama,Bibb County,1.291599
1009_2022-11-01,1009,2022-11-01,Alabama,Blount County,1.835726


### Thoughts on the high-risk solution

The very fact that we were able to train one single model for all the counties at once shows the superiority over statistical time series approach of looking at each individual univariate forecasting (for each county) as independent models. In medium risk solution we had to reduce the scope to train for first 50 counties as we estimated it might take hours to train the models for 3000+ counties

The cherry on the top is that we trained the model for high-risk solution on the best parameters detected!! 

**Performance Evaluation:**

Since the LGBM objective was set to optimize the SMAPE values we found the best performance came on 26th iteration of Optuna Framework where SMAPE was found to be **1.06766**. Given that this is the metric for the entire dataset for all 3000 counties the performance isn't too bad. With more feature engineering and data cleaning we can definitely optimize this model. 